In [39]:
geneinfo = read.table("/publicdata/gencode_v19_20151104/gene_info.tsv", sep = "\t", header = T)

# Genes that change

In [2]:
# Signature genes

out = "/projects/PPC/analysis/ppc_pilot/data/aggr_live_then_integrate/"

iPSC_markers        = read.table(paste(out, "iPSC_markers.txt", sep = ""))
repl_markers        = read.table(paste(out, "repl_markers.txt", sep = ""))
ppc_markers         = read.table(paste(out, "all.ppc_markers.txt", sep = ""))
Endocrine_markers   = read.table(paste(out, "Endocrine_markers.txt", sep = ""))
mesenchymal_markers = read.table(paste(out, "mesenchymal_markers.txt", sep = ""))
endothelial_markers = read.table(paste(out, "endothelial_markers.txt", sep = ""))


In [3]:
# Changes over PPC time

out = "/projects/PPC/analysis/ppc_pilot/data/aggr_live_then_integrate/"

lm_fit_coefs = read.table(paste(out, "ppc_fit_coefs_frazer_ppc_allexpressedgenes.txt", sep = ""))
lm_fit_coefs_sig = lm_fit_coefs[lm_fit_coefs$q_value < 0.05, ]

mfuzz = read.table(paste(out, "mfuzz_k10_b4_memberships.txt", sep = ""))

# Top 5

In [33]:
g2 = mfuzz[mfuzz$maxCol == 2 & mfuzz$real == TRUE,]
g6 = mfuzz[mfuzz$maxCol == 6 & mfuzz$real == TRUE,]

In [214]:
n = 100

iPSC_sigs        = data.frame(celltype = "iPSC",        gene_name = rownames(iPSC_markers[iPSC_markers$p_val_adj < 0.05, ])[1:n],                     qval = iPSC_markers[iPSC_markers$p_val_adj < 0.05, "p_val_adj"][1:n]              )
PPC_sigs         = data.frame(celltype = "all_PPC",     gene_name = rownames(ppc_markers[ppc_markers$p_val_adj < 0.05, ])[1:n],                       qval = ppc_markers[ppc_markers$p_val_adj < 0.05, "p_val_adj"][1:n]               )
repl_sigs        = data.frame(celltype = "repl",        gene_name = rownames(repl_markers[repl_markers$p_val_adj < 0.05, ])[1:n],                     qval = repl_markers[repl_markers$p_val_adj < 0.05, "p_val_adj"][1:n]              )
endrocrine_sigs  = data.frame(celltype = "endrocrine",  gene_name = rownames(Endocrine_markers[Endocrine_markers$p_val_adj < 0.05, ])[1:n],           qval = Endocrine_markers[Endocrine_markers$p_val_adj < 0.05, "p_val_adj"][1:n]    )
mesenchyme_sigs  = data.frame(celltype = "mesenchyme",  gene_name = rownames(mesenchymal_markers[mesenchymal_markers$p_val_adj < 0.05, ])[1:n],       qval = mesenchymal_markers[mesenchymal_markers$p_val_adj < 0.05, "p_val_adj"][1:n])
endothelial_sigs = data.frame(celltype = "endothelial", gene_name = rownames(endothelial_markers[endothelial_markers$p_val_adj < 0.05, ])[1:n],       qval = endothelial_markers[endothelial_markers$p_val_adj < 0.05, "p_val_adj"][1:n])
delta_ppc_sigs   = data.frame(celltype = "delta_PPC",   gene_name = lm_fit_coefs_sig[order(-abs(lm_fit_coefs_sig$estimate)), "gene_short_name"][1:n], qval = lm_fit_coefs_sig[order(-abs(lm_fit_coefs_sig$estimate)), "q_value"][1:n])

interesting_genes = rbind(iPSC_sigs, PPC_sigs, repl_sigs, endrocrine_sigs, mesenchyme_sigs, endothelial_sigs, delta_ppc_sigs)

In [228]:
table(rownames(Endocrine_markers) %in% "SEPT6")


FALSE 
  262 

In [215]:
interesting_genes.annotated = merge(interesting_genes, geneinfo, by = "gene_name")

In [216]:
interesting_genes.annotated[interesting_genes.annotated$gene_name == "POU5F1",]

,gene_name,celltype,qval,gene_id,gene_type,chrom,start,end,strand,gene_status,source,level
,<chr>,<chr>,<dbl>,<chr>,<chr>,<chr>,<int>,<int>,<chr>,<chr>,<chr>,<int>
500,POU5F1,iPSC,0,ENSG00000204531.11,protein_coding,chr6,31132118,31148508,-,KNOWN,HAVANA,1


# Alternative poly-Adenylation (APA)

In [217]:
apa = read.table("../Table/results_peak_pvalues.txt", head = T)

In [218]:
interesting_genes.annotated$gene   = unlist(lapply(interesting_genes.annotated$gene_id, function(x){unlist(strsplit(x, "\\."))[[1]]}))
apa$gene               = unlist(lapply(apa$Gene_ID, function(x){unlist(strsplit(x, "\\."))[[1]]}))

In [219]:
apa_interesting = apa[apa$gene %in% interesting_genes.annotated$gene, ]

In [223]:
head(apa_interesting)

,Gene_ID,UTR_ID,Peak_ID,iPSC_PUI,PPC_moderate_1_PUI,PPC_moderate_2_PUI,repl_PUI,mesenchymal_PUI,PPC_early_PUI,PPC_late_PUI,Endocrine_PUI,endothelial_PUI,pval,qval,gene
,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
18,ENSG00000060982.14,ENSG00000060982.14_1,ENSG00000060982.14_1_2,-0.39450994,0.72679958,0.6986161,0.7833603,0.5705053,0.7597748,0.5763753,0.8185598,0.8903217,1.62e-93,3.66e-90,ENSG00000060982
23,ENSG00000060982.14,ENSG00000060982.14_1,ENSG00000060982.14_1_1,-0.52289847,-1.28794700,-1.5102721,-1.5064741,-1.3973143,-1.8455076,-1.5405285,-1.1814402,-1.8201717,4.37e-82,9.89e-79,ENSG00000060982
57,ENSG00000112379.8,ENSG00000112379.8_1,ENSG00000112379.8_1_2,-0.07703726,-1.65774971,-1.4944057,-1.5008627,-1.6552195,-1.7331771,-1.4373448,-2.0568164,-1.3843585,6.24e-54,1.39e-50,ENSG00000112379
67,ENSG00000180616.8,ENSG00000180616.8_1,ENSG00000180616.8_1_5,-2.20245915,0.76017672,1.3141873,0.2234635,-0.5631989,-0.2198292,0.8155728,-1.3678805,0.6666667,1.30e-46,2.89e-43,ENSG00000180616
131,ENSG00000152558.14,ENSG00000152558.14_1,ENSG00000152558.14_1_1,-1.27877684,-1.36158690,-1.2896550,-1.2819808,-1.4032684,-1.3789202,-1.1141434,-1.5990776,-1.1570536,3.82e-34,8.23e-31,ENSG00000152558
181,ENSG00000152558.14,ENSG00000152558.14_1,ENSG00000152558.14_1_2,-0.41443194,-0.02842038,-0.0896379,-0.0896832,-0.2098020,-0.1395303,-0.2018253,-0.1080913,-0.5049769,2.84e-28,5.99e-25,ENSG00000152558


In [220]:
interesting_genes.annotated[interesting_genes.annotated$gene %in% apa_interesting$gene, ]

,gene_name,celltype,qval,gene_id,gene_type,chrom,start,end,strand,gene_status,source,level,gene
,<chr>,<chr>,<dbl>,<chr>,<chr>,<chr>,<int>,<int>,<chr>,<chr>,<chr>,<int>,<chr>
1,A1CF,endrocrine,3.162820e-190,ENSG00000148584.10,protein_coding,chr10,52559168,52645435,-,KNOWN,HAVANA,2,ENSG00000148584
52,BCAT1,mesenchyme,0.000000e+00,ENSG00000060982.10,protein_coding,chr12,24964294,25102393,-,KNOWN,HAVANA,2,ENSG00000060982
56,BIRC5,repl,0.000000e+00,ENSG00000089685.10,protein_coding,chr17,76210266,76221717,+,KNOWN,HAVANA,2,ENSG00000089685
57,BIRC5,iPSC,0.000000e+00,ENSG00000089685.10,protein_coding,chr17,76210266,76221717,+,KNOWN,HAVANA,2,ENSG00000089685
66,C14orf132,endrocrine,3.590598e-199,ENSG00000227051.4,protein_coding,chr14,96505660,96560417,+,KNOWN,HAVANA,1,ENSG00000227051
69,C1orf21,mesenchyme,0.000000e+00,ENSG00000116667.8,protein_coding,chr1,184356191,184598154,+,KNOWN,HAVANA,2,ENSG00000116667
94,CBX5,repl,0.000000e+00,ENSG00000094916.9,protein_coding,chr12,54624723,54673886,-,KNOWN,HAVANA,1,ENSG00000094916
119,CDH6,mesenchyme,0.000000e+00,ENSG00000113361.8,protein_coding,chr5,31193856,31329253,+,KNOWN,HAVANA,2,ENSG00000113361
120,CDH6,all_PPC,0.000000e+00,ENSG00000113361.8,protein_coding,chr5,31193856,31329253,+,KNOWN,HAVANA,2,ENSG00000113361


In [221]:
toString(unique(interesting_genes.annotated[interesting_genes.annotated$gene %in% apa_interesting$gene, ]$gene_id))

[1] "ENSG00000148584.10, ENSG00000060982.10, ENSG00000089685.10, ENSG00000227051.4, ENSG00000116667.8, ENSG00000094916.9, ENSG00000113361.8, ENSG00000147202.13, ENSG00000075420.8, ENSG00000213949.4, ENSG00000112379.8, ENSG00000050405.9, ENSG00000119547.5, ENSG00000166450.8, ENSG00000180616.4, ENSG00000159167.7, ENSG00000196628.9, ENSG00000100234.11, ENSG00000152558.10"